<a href="https://colab.research.google.com/github/CleverAndWitty/nasa-airport-config-runtime/blob/main/PredictRunways.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import google shared drive

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


Install PySpark

In [ ]:
!pip install pyspark
!pip install xgboost

In [ ]:
import pyspark
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.conf import SparkConf
from pyspark import SparkContext

conf = SparkConf().setAppName('RunwayConfigPrediction_TeamTheRedFlags')
sc = SparkContext.getOrCreate(conf = conf)
ohe = pyspark.ml.feature.OneHotEncoder()
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

import plotnine
import xgboost as xgp
xgbParameters = {'tree_method': 'gpu_exact'}

#################################
 ###### STEPS FOR XGBOOST ######
  #############################

#use DMATRIX before inputing matrix to xgb
#create dictionary of parameters to pass in eg: 
### params = {"objective":"binary:logistic","max_depth":4} 
#Make a cv test using xgb.cv eg:
## xgb.cv(dtrain = DMATRIX, params = params, nfold = 4, num_boost_round=10, metrics="error", as_pandas=True)



import datetime



Create Data Frames

In [182]:

#from typing_extensions import ParamSpecArgs
strFilePath = "/content/drive/Shareddrives/CIS3200GroupProject/DecompressedByAirport/k"

filestrAirportConfig = "_airport_config.csv"
filestrArrivalRunway = "_arrival_runway.csv"
filestrDepartureRunway = "_departure_runway.csv"
filestrEstTimeDeparture = "_etd.csv"
filestrFirstPosition = "_first_position.csv"
filestrLamp = "_lamp.csv"
filestrMFSRunwayArrTime = "_mfs_runway_arrival_time.csv"
filestrMFSRunwayDptTime = "_mfs_runway_departure_time.csv"
filestrMFSStandArrTime = "_mfs_stand_arrival_time.csv"
filestrMFSStandDeptTime = "_mfs_stand_departure_time.csv"
filestrTBFMSchedRunwayArrTime = "_tbfm_scheduled_runway_arrival_time.csv"
filestrTFMEstRunwayArrTime = "_tfm_estimated_runway_arrival_time.csv"

arrayAirportString = [
                      "atl"
                      ,"clt"
                      ,"den"
                      ,"dfw"
                      ,"dfw"
                      ,"jfk"
                      ,"mem"
                      ,"mia"
                      ,"ord"
                      ,"phx"
                      ,"sea"
                      ]

arrayFileString = [
  filestrAirportConfig,
  filestrArrivalRunway,
  filestrDepartureRunway,
  filestrEstTimeDeparture,
  filestrFirstPosition,
  filestrLamp,
  filestrMFSRunwayArrTime,
  filestrMFSRunwayDptTime,
  filestrMFSStandArrTime,
  filestrMFSStandDeptTime,
  filestrTBFMSchedRunwayArrTime,
  filestrTFMEstRunwayArrTime
  ]
DFDictionary = {}
for AirportCode in arrayAirportString:
  DFDictionary[AirportCode] = {}
  for FileString in arrayFileString:
    Filepath = str(strFilePath + AirportCode + "/k" + AirportCode + FileString)
    #WorkingFile = spark.read.csv()
    
    dfParameters = {
        'path': Filepath, 
        'header':True,
        'enforceSchema':False
        }

    dfSchema = StructType()
    isCreateDataFrame = 0

    if FileString == filestrAirportConfig:
      strFileShorthand = "ArptConfig"
      dfSchema.add("timestamp", TimestampType(), False)
      dfSchema.add("airport_config", StringType(), True)
      isCreateDataFrame = 1
    elif FileString == filestrArrivalRunway:
      strFileShorthand = "ArrRnwy"
      dfSchema.add("gufi", StringType(), True)
      dfSchema.add("timestamp", TimestampType(), False)
      dfSchema.add("arrival_runway", StringType(), True)
      isCreateDataFrame = 1
    elif FileString == filestrDepartureRunway:
      strFileShorthand = "DeptRnwy"
      dfSchema.add("gufi", StringType(), True)
      dfSchema.add("timestamp", TimestampType(), False)
      dfSchema.add("departure_runway", StringType(), True)
      isCreateDataFrame = 1
    elif FileString == filestrLamp:
      strFileShorthand = "Lamp"
      dfSchema.add("timestamp", TimestampType(), False)
      dfSchema.add("forecast_timestamp", TimestampType(), True)
      dfSchema.add("temperature", DecimalType(3,0), True)
      dfSchema.add("wind_direction", DecimalType(3,0), True)
      dfSchema.add("wind_speed", DecimalType(3,0), True)
      dfSchema.add("wind_gust", DecimalType(3,0), True)
      dfSchema.add("cloud_ceiling", DecimalType(1,0), True)
      dfSchema.add("visibility", DecimalType(1,0), True)
      dfSchema.add("cloud", StringType(), True)
      dfSchema.add("lightning_prob", StringType(), True)
      dfSchema.add("precip", BooleanType(), True)
      isCreateDataFrame = 1
      
    #"""THESE FILES ARE NOT NEEDED FOR TRAINING SET
    elif FileString == filestrEstTimeDeparture:
      strFileShorthand = "ETD"
      dfSchema.add("gufi", StringType(), True)
      dfSchema.add("timestamp", TimestampType(), True)
      dfSchema.add("estimated_runway_departure_time", TimestampType(), True)
    elif FileString == filestrFirstPosition:
      strFileShorthand = "FrstPos"
      dfSchema.add("gufi", StringType(), True)
      dfSchema.add("timestamp", TimestampType(), True)
    elif FileString == filestrMFSRunwayArrTime:
      strFileShorthand = "MfsRnwyArrTime"
      dfSchema.add("gufi", StringType(), True)
      dfSchema.add("timestamp", TimestampType(), True)
    elif FileString == filestrMFSRunwayDptTime:
      strFileShorthand = "MfsRnwyDptTime"
      dfSchema.add("gufi", StringType(), True)
      dfSchema.add("timestamp", TimestampType(), True)
    elif FileString == filestrMFSStandArrTime:
      strFileShorthand = "MfsStndArrTime"
      dfSchema.add("gufi", StringType(), True)
      dfSchema.add("timestamp", TimestampType(), True)
    elif FileString == filestrMFSStandDeptTime:
      strFileShorthand =  "MfsStndDeptTime"
      dfSchema.add("gufi", StringType(), True)
      dfSchema.add("timestamp", TimestampType(), True)
    elif FileString ==filestrTBFMSchedRunwayArrTime:
      strFileShorthand = "TbfmSchdRnwyArrTime"
      dfSchema.add("gufi", StringType(), True)
      dfSchema.add("timestamp", TimestampType(), True)
      dfSchema.add("scheduled_runway_arrival_time", TimestampType(), True)
    elif FileString == filestrTFMEstRunwayArrTime:
      strFileShorthand = "TfmEstRnwyArrTime"
      dfSchema.add("gufi", StringType(), True)
      dfSchema.add("timestamp", TimestampType(), True)
      dfSchema.add("estimated_runway_arrival_time", TimestampType(), True)
    #END FILES NOT NEEDED FOR TRAINING SET"""

    if isCreateDataFrame == 1:
      df = spark.read.format("csv").option("header",True).schema(dfSchema).load(Filepath)
      #df = spark.read.csv(**dfParameters) #Use this when you've added a new file and need to see the column names
      DFDictionary[AirportCode][strFileShorthand] = df
    
    
      
    #print(Filepath)
    #print("For Airport " + AirportCode, "\n Completed Loading:", strFileShorthand)

[OPTIONAL] Display and test dictionary of dataframes

In [ ]:
for key1 in DFDictionary:
  for key2 in DFDictionary[key1]:
    newDF = DFDictionary[key1][key2]
    dtype = newDF.dtypes
    print("Airport:" + str(key1) + "\nFileShortHand:" + str(key2) + str(dtype))
    newDF.show(truncate=False)


In [207]:
pivotdfConfig     = DFDictionary['mia']['ArptConfig'].groupby("timestamp").pivot("airport_config").count()#,"DepartureRunways", "ArrivalRunways").count()
pivotdfDeparture  = DFDictionary['mia']['ArptConfig'].groupby("timestamp").pivot("DepartureRunways").count()#,"DepartureRunways", "ArrivalRunways").count()
pivotdfArrival    = DFDictionary['mia']['ArptConfig'].groupby("timestamp").pivot("ArrivalRunways").count()#,"DepartureRunways", "ArrivalRunways").count()


pivotdfDeparture =  pivotdfDeparture.withColumnRenamed('timestamp','deptTimestamp')
pivotdfArrival   =  pivotdfArrival.withColumnRenamed('timestamp','arrvTimestamp')   

pivotdfConfig.createOrReplaceTempView("Config")
pivotdfDeparture.createOrReplaceTempView("Departure")
pivotdfArrival.createOrReplaceTempView("Arrival")

query = "Select * FROM Config as a Join Departure as b on 'a.timestamp' = 'b.deptTimestamp' JOIN Arrival as c on 'a.timestamp' = 'c.arrvTimestamp'"

pivotdf = spark.sql(query)

#Got this from https://stackoverflow.com/questions/51381038/filling-not-null-values-as-1-in-pyspark-dataframe
#First define the sequence of the columns
ConfigColumn  = pivotdfConfig.columns
DeptColumn    = pivotdfDeparture.columns
ArvlColumn    = pivotdfArrival.columns

#Make selections with replacements for each value found that isn't the 1st col
ConfigSelections  = [when(col(column).isNull(),lit(0)).otherwise(lit(1)).alias(column) for column in ConfigColumn[1:]]
DeptSelections    = [when(col(column).isNull(),lit(0)).otherwise(lit(1)).alias(column) for column in ConfigColumn[:(len(ConfigColumn)-1)]]
ArrvSelections    = [when(col(column).isNull(),lit(0)).otherwise(lit(1)).alias(column) for column in ConfigColumn[:(len(ConfigColumn)-1)]]

pivotdf.show(truncate=False)
pivotdfConfig.select(ConfigSelections).show(truncate=False)
pivotdfDeparture.select(DeptSelections).show(truncate=False)
pivotdfArrival.select(ArrvSelections).show(truncate=False)

+---------+-----------------------+-------------------------+----------------------+--------------------+----------------------+-------------------+----------------------+-------------------+-------------------+----------------+---------------------+-----------------------+------------------+-------------------+-------------------+----------------+----------------------+-----------------+-------------------+----------------+-------------------+----------------+------------+---------------+-------------------+----------------+-------------+----------------------------+-------------------------+----------------------------+-------------------------------+----------------------------------+----------------------------+------------------------+---------------------------+---------------------------+-----------------------+------------------+----------------------+-------------------------+----------------------------+----------------------------+---------------------+----------------------

AnalysisException: ignored

Functions used in data prep 

In [154]:
#Does null handling for Lightning Probability Column
def LightningNoneCheck(StringInput):
  if StringInput != "None":
    return StringInput
  elif StringInput == None:
    StringOutput = "N"
    return StringOutput
  elif StringInput is NullType:
    StringOutput = "N"
    return StringOutput

#Parses Airport Runway Configuration into a tuple of Departure and Arrival 
##runways
def AirportConfigDeptArv(strAirportRunwayConfig):
  for i in range(len(strAirportRunwayConfig)):
    CurrentCharacter = strAirportRunwayConfig[i]
    if CurrentCharacter == 'A':
      DepartureString = strAirportRunwayConfig[2:(i-1)]
      ArrivalString = strAirportRunwayConfig[(i+2):(len(strAirportRunwayConfig))]
      return(DepartureString, ArrivalString)

#Maps all possible runway configurations for a column
#Reduces by count
#Remaps by eliminating the counts
#Outputs rdd of all unique entries in column

def MapReduceOneHotArptConfig(AirportConfigDataFrame, Column):
  
  if Column == 'airport_config':
    ListOutput = \
    AirportConfigDataFrame.na.drop().\
    rdd.map(lambda row : (row.airport_config,1))
  elif Column == 'DepartureRunways':
    ListOutput = \
    AirportConfigDataFrame.na.drop().\
    rdd.map(lambda row : (row.DepartureRunways,1))
  elif Column == 'ArrivalRunways':
    ListOutput = \
    AirportConfigDataFrame.na.drop().\
    rdd.map(lambda row : (row.ArrivalRunways,1))
  
  ListOutput = \
    ListOutput.\
    reduceByKey(lambda a,b: a+b)

  ListOutput = \
    ListOutput.\
    map(lambda a: a[0])
  
  return ListOutput.collect()


def WindowTimeStamp(Dataframe):
  pass

[DATA PREP]

Only run once
Null Handling
Timestamp Merger

In [183]:
from pyspark.sql.column import Column
ArptConfigEncoder = {}

for airport in DFDictionary:  
  ArptConfigEncoder[airport] = {}
  for fileshort in DFDictionary[airport]:
    if fileshort == 'ArptConfig':
      #Create new columns with individual runways used (departure & arrival)
      DFDictionary[airport]['ArptConfig'] = DFDictionary[airport]['ArptConfig'].\
        na.drop().rdd.map(lambda row: row + AirportConfigDeptArv(row.airport_config))
      DFDictionary[airport][fileshort] = spark.createDataFrame(
          DFDictionary[airport][fileshort]).withColumnRenamed('_1','timestamp').\
      withColumnRenamed('_2', 'airport_config').\
      withColumnRenamed('_3', 'DepartureRunways').\
      withColumnRenamed('_4', 'ArrivalRunways').\
      withColumn('DepartureRunways', explode(split('DepartureRunways', '_'))).\
      withColumn('ArrivalRunways', explode(split('ArrivalRunways', '_')))

#withColumnRenamed('_3', 'isExists').\

      #Create 3 rdd's to be used for one-hot encoding output columns
      ArptConfigEncoder[airport]['ConfigOutput'] = MapReduceOneHotArptConfig(
          DFDictionary[airport][fileshort] , 'airport_config'
      )
      ArptConfigEncoder[airport]['DepartureOutput'] = MapReduceOneHotArptConfig(
          DFDictionary[airport][fileshort] , 'DepartureRunways'
      )
      ArptConfigEncoder[airport]['ArrivalOutput'] = MapReduceOneHotArptConfig(
          DFDictionary[airport][fileshort] , 'ArrivalRunways'
      )


    if fileshort == 'Lamp':
      DFDictionary[airport][fileshort] = DFDictionary[airport][fileshort].\
      fillna("N", subset = "lightning_prob")
      DFDictionary[airport][fileshort] = DFDictionary[airport][fileshort].\
      fillna("CL", subset = "cloud")
      DFDictionary[airport][fileshort] = DFDictionary[airport][fileshort].\
      fillna(8, subset = "cloud_ceiling")
      DFDictionary[airport][fileshort] = DFDictionary[airport][fileshort].\
      withColumn("lightning_prob", when(DFDictionary[airport][fileshort].\
                                        lightning_prob == None, "N").\
                 otherwise(DFDictionary[airport][fileshort].lightning_prob))
      DFDictionary[airport][fileshort].createOrReplaceTempView("DataFrame")
      query = "SELECT timestamp  ,forecast_timestamp  ,temperature  ,wind_direction  ,wind_speed  ,wind_gust  ,cloud_ceiling  ,visibility  ,cloud  ,lightning_prob  ,precip  FROM (SELECT timestamp   ,forecast_timestamp  ,temperature  ,wind_direction  ,wind_speed  ,wind_gust  ,cloud_ceiling  ,visibility  ,cloud  ,lightning_prob  ,precip,ROW_NUMBER() OVER(PARTITION BY timestamp ORDER BY forecast_timestamp asc) as row_number FROM DataFrame) WHERE row_number = 1"
      DFDictionary[airport][fileshort] = spark.sql(query).na.drop()

    """DFDictionary[airport][fileshort].timestamp = (
        DFDictionary[airport][fileshort]
    )"""

    DFDictionary[airport][fileshort] = DFDictionary[airport][fileshort].withColumn('isExists', lit(1))


In [162]:
for value in ArptConfigEncoder:
  print(value)
  for dfish in ArptConfigEncoder[value]:
    print(dfish + ":\n\t")
    print(ArptConfigEncoder[value][dfish])

"""
ohe.setInputCols(['airport_config', 'DepartureRunways', 'ArrivalRunways'])
ohe.setOutputCols([ArptConfigEncoder['mia']['ConfigOutput']
                   ,ArptConfigEncoder['mia']['DepartureOutput']
                   ,ArptConfigEncoder['mia']['ArrivalOutput']])
"""

atl
ConfigOutput:
	
['D_8R_9L_A_10_8L_9R', 'D_8L_9L_A_10_8L_9R', 'D_26L_27R_8R_9L_A_26R_27L_28', 'D_26L_27R_A_26R_27L_28', 'D_26L_27R_A_26R_27R_28', 'D_26R_27R_A_26R_27R_28', 'D_26R_27R_A_26R_27L_28', 'D_26L_26R_27R_A_26R_27L_28', 'D_8R_9L_A_10_8R_9R', 'D_8L_9L_A_10_8L_9L', 'D_8L_8R_9L_A_10_8L_9R', 'D_9L_A_10_9R', 'D_9L_A_10_8L_9R', 'D_8R_9L_A_10_8L', 'D_9L_A_10', 'D_27R_A_28', 'D_26L_A_26R_27L_28', 'D_26L_27R_A_26R_27L', 'D_27R_A_26R_27L_28', 'D_27R_A_27L_28', 'D_26L_27R_A_26L_27L_28', 'D_27R_28_A_27L_28', 'D_26L_27R_28_A_26R_27L_28', 'D_28_8R_9L_A_10_8L_9R', 'D_26L_27R_A_26R_28', 'D_8R_9L_A_10_8L_9L', 'D_26L_27R_8R_9L_A_10_8L_9R', 'D_8R_9L_A_8L_9R', 'D_8R_A_10_8L_9R', 'D_26L_27R_9L_A_26R_27L_28', 'D_26L_27L_A_26R_27L_28', 'D_26L_28_A_26R_27L_28', 'D_26L_27R_A_27L_28', 'D_8R_9L_A_10_9R', 'D_26L_27L_27R_A_26R_27L_28', 'D_26L_27R_A_26L_27L', 'D_26L_27L_A_26L_27L_28', 'D_8R_9R_A_10_8L_9R', 'D_8R_9L_A_10_8R_9L', 'D_26R_27R_A_26R_27L', 'D_8R_9R_A_10_8R_9R', 'D_26R_27L_A_26R_27L_28', 'D_8L_

"\nohe.setInputCols(['airport_config', 'DepartureRunways', 'ArrivalRunways'])\nohe.setOutputCols([ArptConfigEncoder['mia']['ConfigOutput']\n                   ,ArptConfigEncoder['mia']['DepartureOutput']\n                   ,ArptConfigEncoder['mia']['ArrivalOutput']])\n"

In [ ]:
DFDictionary['mia']['ArptConfig'].show(truncate=False)

+-------------------+-------------------+----------------+--------------+
|timestamp          |airport_config     |DepartureRunways|ArrivalRunways|
+-------------------+-------------------+----------------+--------------+
|2020-11-01 00:23:00|D_8L_8R_9_A_8L_8R_9|8L              |8L            |
|2020-11-01 00:23:00|D_8L_8R_9_A_8L_8R_9|8L              |8R            |
|2020-11-01 00:23:00|D_8L_8R_9_A_8L_8R_9|8L              |9             |
|2020-11-01 00:23:00|D_8L_8R_9_A_8L_8R_9|8R              |8L            |
|2020-11-01 00:23:00|D_8L_8R_9_A_8L_8R_9|8R              |8R            |
|2020-11-01 00:23:00|D_8L_8R_9_A_8L_8R_9|8R              |9             |
|2020-11-01 00:23:00|D_8L_8R_9_A_8L_8R_9|9               |8L            |
|2020-11-01 00:23:00|D_8L_8R_9_A_8L_8R_9|9               |8R            |
|2020-11-01 00:23:00|D_8L_8R_9_A_8L_8R_9|9               |9             |
|2020-11-01 00:27:00|D_8L_8R_9_A_8L_8R_9|8L              |8L            |
|2020-11-01 00:27:00|D_8L_8R_9_A_8L_8R

In [ ]:
OutputColsStringList = ArptConfigEncoder['mia']['ConfigOutput'].collect()
for i in range(len(OutputColsStringList)):
  OutputColsStringList[i] = str(OutputColsStringList[i])
  
ohe.setInputCols(["airport_config"])
#ohe.setOutputCols(["Output"])
ohe.setOutputCols(OutputColsStringList)
model = ohe.fit(DFDictionary['mia']['ArptConfig'])
newdf = model.transform(DFDictionary['mia']['ArptConfig'])
newdf.show(truncated=False)


IllegalArgumentException: ignored

[OPTIONAL] Used to make output for Nitya

In [ ]:
filpath = "/content/drive/Shareddrives/CIS3200GroupProject/NityaCorrectedDF/"

PdDFLampDictionary = {}
for AirportCode in DFDictionary:
  #DFDictionary[AirportCode] = {}
  #for FileString in arrayFileString:
  filpath += (AirportCode + "Lamp.csv")
  PdDFLampDictionary[AirportCode] = DFDictionary[AirportCode]["Lamp"].toPandas()
  print(AirportCode)
  PdDFLampDictionary[AirportCode].to_csv(filpath)

[WORK IN PROGRESS]
The next few code blocks are for testing unformed ideas

In [ ]:
thing = (
    DFDictionary['mia']['ArptConfig'].groupby(window("timestamp", "15 minutes")).agg(avg(""))
)
thing.collect()

In [ ]:
#DFDictionary[airport][fileshort].filter(col("timestamp").isNull()).show()
"""
Normalize Columns
withColumn("lightning_prob", when(DFDictionary[airport][fileshort].\
                                  lightning_prob == None, "N").\
            otherwise(DFDictionary[airport][fileshort].lightning_prob))
"""
splitmap = DFDictionary['atl']['Lamp'].na.drop().rdd.map(lambda r: (r.wind_direction,1))
splitmap = splitmap.reduceByKey(lambda a,b: a+b)
splitmap.sortBy(lambda x: -x[0]).collect()

#DFDictionary['mia']['Lamp'].rdd.collect()
#splitmap = DFDictionary['mia']['Lamp'].rdd.map(lambda r: LightningNoneCheck(r.lightning_prob))


#Lamp.lightning_prob categorized by L N none M H
#Lamp.cloud categorized by BK CL Few OV SC

In [ ]:
splitmap = DFDictionary['ord']['ArptConfig'].na.drop().rdd.map(lambda r: (r.airport_config,1))
splitmap = splitmap.reduceByKey(lambda a,b: a+b)
splitmap.sortBy(lambda x: -x[1]).collect()

In [ ]:
from pathlib import Path
from typing import Sequence, Tuple

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

Make Pandas Data Frames

In [ ]:
PdDFACDictionary = {}
for AirportCode in DFDictionary:
  PdDFACDictionary[AirportCode] = DFDictionary[AirportCode]["ArptConfig"].toPandas()
  print("Completed Converting rdd: \n\t" + AirportCode)

atl
clt
den
dfw
jfk
mem
mia
ord
phx
sea


In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime
from plotnine import ggplot, aes, geom_freqpoly, xlim


for (AP1, AC), (AP2, WC) in zip(PdDFACDictionary.items(), PdDFLampDictionary.items()):
  # Set figure size for the notebook
  plt.rcParams["figure.figsize"]=15,9
  sns.set(style="whitegrid")
  first_monthAC = AC.loc[(AC['timestamp'] >='2020-11-01') &
                             (AC['timestamp'] <='2020-12-01')]
                             # Without transparency
  sns.kdeplot(data=first_monthAC, x="timestamp", hue="airport_config", fill=True, common_norm=False, alpha=0.4, multiple='stack')
  # plt.title('{} First Month Airport Config'.format(AP1))
  fig, ax = plt.subplots()
  fig.autofmt_xdate()
  sns.set(xlim = (datetime.date(2020, 11, 1), datetime.date(2020, 12, 1)))
  plt.title('{} First Month Airport Config'.format(AP1))
  plt.show()
  first_monthWC = WC.loc[(WC['forecast_timestamp'] >='2020-11-01') &
                             (WC['forecast_timestamp'] <='2020-12-01')]
  plt.scatter(first_monthWC['forecast_timestamp'], 
           first_monthWC['wind_speed'], 
           c = first_monthWC['wind_direction'],
           cmap = 'magma')
  plt.colorbar()
  plt.title('{} First Month Wind Direction'.format(AP2))
  plt.show()
  plt.figure(figsize=(15, 9)) 
  # first_month_shalf = WC.loc[(WC['forecast_timestamp'] >='2020-11-02 21:30:00') &
  #                            (WC['forecast_timestamp'] <='2020-11-02 23:30:00')]
  # plt.plot(first_month_shalf['forecast_timestamp'], first_month_shalf['wind_direction'], 'o', color = WC['timestamp'])
  # plt.title('{} Second Half of First Month Wind Direction'.format(AP))
  # plt.show()
  # plt.figure(figsize=(15, 9))



# phx_WC1 = phx_WC[phx_WC['timestamp'] <'2021-01-01']
# plt.figure(figsize=(15, 9))
# plt.plot(phx_WC1['timestamp'], phx_WC1['wind_direction'], kind = "point", color = 'b')
# plt.show()
# plt.figure(figsize=(15, 9))

# plt.plot(phx_WC1['timestamp'], phx_WC1['wind_speed'])
# plt.show()
# plt.figure(figsize=(15, 9))

# plt.plot(phx_WC1['timestamp'], phx_WC1['wind_gust'])
# plt.show()

NameError: ignored

**NASA AIRPORT CONFIGURATION BENCHMARK RECONSTRUCTION**

# **READ THE TRAINING LABELS**

In [ ]:
# open_train_labels = pd.read_csv(
#     DATA_PATH / "open_train_labels.csv.bz2", parse_dates=["timestamp"]
# )
# open_train_labels.shape

In [ ]:
# open_train_labels.head()

# **GET A SAMPLE FRAME**

In [ ]:
# sample_frame_groups = open_train_labels.groupby(["airport", "timestamp", "lookahead"], sort=False)